# Evolutionary Computation - Assignment 2: Greedy Regret Heuristics

Bartosz Stachowiak 148259<br>
Andrzej Kajdasz 148273

## 1. Problem Statement

There were are columns of integers representing nodes. Each row corresponds to a node and contains its x and y coordinates in a plane, as well as a cost associated with the node. There were 4 such data sets each consisting of 200 rows (each representing a single node).

Problem to solve is to choose precisely 50% of the nodes (rounding up if there is an odd number of nodes) and create a Hamiltonian cycle (a closed path) using this subset of nodes. The goal is to minimize the combined total length of the path and the total cost of the selected nodes.

To calculate the distances between nodes, the Euclidean distance formula was used and then round the results to the nearest integer. As suggested, the distances between the nodes were calculated after loading the data and placed in a matrix, so that during the subsequent evaluation of the problem, it was only necessary to read these values which reduced the cost of the operation of the algorithm.

Two approach were used to solve the problem:

- Greedy 2-regret heuristics.
- Greedy heuristics with a weighted sum criterion

### Important note
In case of greedy heuristics with a weighted sum criterion algorithm we are using a naming convention "Weighted greedy heuristics (x)", 

## 2. Pseudocode of all implemented algorithms
>Note that in our implementation, distance matrix is not passed as an argument, but a class member, calculated before the algorithm is run.\
>For simplification, here we pass it as an argument.

In our implementation, we noticed that by transforming the formula for the algorithm using weights, we can both optimize the number of calculations and - with the choice of appropriate weights - obtain a 2-regret algorithm:

Let:
- $m_1$ - minimum increase
- $m_2$ - second minimum increase
- $w$ - weight factor favoring the first minimum increase

Then:
- 2-regret is the maximization of $m_2 - m_1$
- weighted sum is the maximization of $(2 - w) * m_2 - w * m_1$

Hence we can use the same algorithm for both cases, by passing the appropriate weights.
- For standard **2-regret**, we pass $w = 1$
- For $w = 2$ we get maximization of $-2*m_1$ which is equivalent to minimization of $m_1$ - which boils down to the vanilla **Greedy Cycle Algorithm**

Thus we provide pseudocode only for the weighted version.



```
function greedy_heuristics_with_weighted_criterion(nodes, start_index, target_size, distance_matrix, weight_of_min_increase):

    solution := [start_index]
    visited := [false, false, ..., false]
    visited[start_index] := true

    // while solution is not complete, add nodes
    while len(solution) < target_size:

        best_weighted_criterion := -infinity
        best_index := -1
        insert_point := -1

        if len(solution) == 1:
            // find cheapest neighbor
            min_increase = infinity
            for i in 0..len(nodes) - 1:
                if visited[i]:
                    continue
                if nodes[i].weight < min_increase:
                    min_increase := nodes[i].cost
                    best_index := i

            solution.append(best_index)
            visited[best_index] := true
            continue

        for i in 0..len(nodes) - 1:
            if visited[i]:
                continue
            
            added_idx = -1;
            min_increase = infinity
            second_min_increase = infinity
            
            // find two related cheapest nodes
            for j in 0..len(solution) - 1:

                next_idx := (j + 1) % len(solution)
                added_distance := distance_matrix[solution[j]][i] + distance_matrix[i][solution[next_idx]]
                removed_distance := distance_matrix[solution[j]][solution[next_idx]]
                increase := added_distance + nodes[i].weight - removed_distance

                if increase <= min_increase:
                    second_min_increase := min_increase
                    min_increase := increase
                    added_index := j
                else if increase < second_min_increase:
                    second_min_increase := increase
            
            //calculate regret and comapre to max regret
            weighted_criterion := (2 - weight_of_min_increase) * second_min_increase - weight_of_min_increase * min_increase
            if weighted_criterion > best_weighted_criterion:
                best_weighted_criterion := weighted_criterion
                insert-point := i
                best_index := added_idx
                

        solution.insert(insert_point, best_index)
        visited[min_index] := true

    return solution
```
    

## 3. Results of the computational experiments

For consistency with the task description, we use the following naming convention for the weighted algorithm:
- *Greedy 2-regret heuristics* - for $w = 1$
- *Weighted greedy heuristics (x)* - for $w = x + 1$ (as it denotes combination of 2-regret and best change of the objective function; $x$ is a weight of best change of the objective function and $(1-x)$ is a weight of 2-regret value)

This notation is also used in the conclusions section.

### 3.1. Code for visualization of the results

In [1]:
import pathlib
import itertools

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from common import *

In [ ]:
DATA_FOLDER = './data/'
RESULT_FOLDER = f'{DATA_FOLDER}results/'
INSTANCE_FOLDER = f'{DATA_FOLDER}tsp_instances/'

SOLVERS = {
    'r': "Random",
    'n': "Nearest Neighbor",
    'g': "Greedy Cycle",
    "d-0.5": "Weighted greedy heuristics (-0.5)",
    'd-1': "Greedy 2-regret heuristics",
    'd-1.25': "Weighted greedy heuristics (0.25)",
    'd-1.5': "Weighted greedy heuristics (0.5)",
    'd-1.75': "Weighted greedy heuristics (0.75)",
    'd-2': "Weighted greedy heuristics (1.0)",
}
NUM_NODES = 200

instance_files = [path for path in pathlib.Path(INSTANCE_FOLDER).iterdir() if path.is_file()]
instance_names = [path.name[:4] for path in instance_files]

In [ ]:
instances_data = {
    name: read_instance(f'{INSTANCE_FOLDER}{name}.csv')
    for name in instance_names
}

In [ ]:
instances_solvers_pairs = itertools.product(instances_data.keys(), SOLVERS.keys())

all_results = {}
all_costs = {}
all_stats = {}

for instance, solver in instances_solvers_pairs:
    all_results[instance] = all_results.get(instance, {})
    all_costs[instance] = all_costs.get(instance, {})
    all_stats[instance] = all_stats.get(instance, {})
    costs = []
    paring_results = []
    for idx in range(NUM_NODES):
        solution, cost, time = read_solution(f'{RESULT_FOLDER}{instance}-{solver}-{idx}.txt')
        paring_results.append(solution)
        costs.append(cost)
    all_results[instance][solver] = np.array(paring_results)
    all_costs[instance][solver] = np.array(costs)
    all_stats[instance][solver] = {
        'mean': np.mean(costs),
        'std': np.std(costs),
        'min': np.min(costs),
        'max': np.max(costs),
    }

In [ ]:
mean_df = pd.DataFrame(all_stats).T
max_df = pd.DataFrame(all_stats).T
min_df = pd.DataFrame(all_stats).T

for column in SOLVERS.keys():
    mean_df[column] = mean_df[column].apply(lambda x: f'{x["mean"]:.0f} ± {x["std"]:.0f}')
    max_df[column] = max_df[column].apply(lambda x: x['max'])
    min_df[column] = min_df[column].apply(lambda x: x['min'])

for df in [mean_df, max_df, min_df]:
    df.rename(columns=SOLVERS, inplace=True)

### 3.2. Visualizations and statistics for all dataset-algorithm pairs

In tabular form we present the Mean, Standard Deviation, Minimum and Maximum of the results of the algorithms for each dataset.

In [ ]:
print("Mean and std of the costs:")
mean_df


In [ ]:
fig, axs = plt.subplots(1, len(instances_data.keys()), figsize=(15, 5), sharey=True)

for idx, instance in enumerate(instances_data.keys()):
    if idx == 0:
        axs[idx].set_ylabel('Cost')
    axs[idx].set_title(instance)
    axs[idx].violinplot(
        [all_costs[instance][solver] for solver in SOLVERS.keys()],
        showmeans=True,
    )
    axs[idx].set_xticks(range(1, len(SOLVERS.keys()) + 1))
    axs[idx].set_xticklabels(SOLVERS.values(), rotation=45, ha='right')

plt.suptitle('Distribution of the costs')
plt.show()

In [ ]:
print("Max of the costs:")
max_df

In [ ]:
print("Min of the costs:")
min_df

In [ ]:
fig, axs = plt.subplots(1, len(instances_data.keys()), figsize=(15, 5), sharey=True)
bar_width = 0.35

for idx, instance in enumerate(instances_data.keys()):
    if idx == 0:
        axs[idx].set_ylabel('Cost')
    axs[idx].set_title(instance)

    axs[idx].bar(
        np.arange(len(SOLVERS.keys())),
        max_df.loc[instance].values,
        width=bar_width,
        label='Maximal cost',
    )
    axs[idx].bar(
        np.arange(len(SOLVERS.keys())) + bar_width,
        min_df.loc[instance].values,
        width=bar_width,
        label='Minimal cost',
    )

    axs[idx].set_xticks(np.arange(len(SOLVERS.keys())) + bar_width / 2)
    axs[idx].set_xticklabels(SOLVERS.values(), rotation=45, ha='right')

plt.suptitle('Maximal cost')
plt.legend()
plt.show()

## 4. Best solutions for all datasets and algorithms

To more easily compare the results, we present the best solutions for each dataset side by side.

The weight of each node is denoted both by its size and color. The bigger and brighter the node, the higher its weight.

In [ ]:
for idx, instance in enumerate(instances_data.keys()):
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for solver_idx, solver in enumerate(["d-1","d-1.25","d-1.5","d-1.75"]):
        best_instance_idx = np.argmin(all_costs[instance][solver])
        plot_solution_for_instance(instances_data[instance], all_results[instance][solver][best_instance_idx], axs[solver_idx])
        axs[solver_idx].set_title(f'{SOLVERS[solver]}: {all_costs[instance][solver][best_instance_idx]:.0f}')
    fig.suptitle(f'Instance {instance}', fontsize=16, y=1.05)
plt.show()


## 5. Source Code

[GitHub](https://github.com/Tremirre/ECP)

## 6. Conclusions

Analyzing the results and visualizations, one can come to several conclusions about the algorithms used in the task:

- Greedy heuristics with a weighted sum criterion is an algorithm that combines greedy cycle and greedy 2-regret heuristics. It should be noted that if the weight of the best change of the objective function is 0 then the algorithm will start acting like greedy 2-regret heuristics, and when the weight of 2-regret factor is 0 then the algorithm will act like greedy cycle what is confirmed by the results we obtained.
  
- During experiments with the greedy heuristics with a weighted sum criterion, the following cases were tested:
    - Equal weights (results represent by *Weighted greedy heuristics (0.5)*):
       - Works much better than greedy 2-regret heuristics
       - In comparison to Random, Nearest Neighbor and Greedy Cycle based on obtained minimum cost algorithm works better for all dataset with only one exception (for TSPA greedy cycle is slightly better). In case of avarage cost works also better but also with one exception (for TSPD mean value of Nearest Neighbor is lower). 
    - 2-regret weight bigger than weight best change of the objective function "Weighted greedy heuristics (0.25)":
    - Works slightly worse for TSPA, TSPB and TSPC than equal weights, but for TSPD it has the best result in comparison to all other algorithms. 
    - 2-regret weight smaller than weight best change of the objective function "Weighted greedy heuristics (0.75)":
       - Better than other algorithms for TSPB
       - Very similar results to Greedy Cycle what is obvious based on first conclusion
<br><br>
- Referring to the second conclusion, it can be summarized that greedy heuristics with a weighted sum criterion algorithm enables to obtain better results than algorithms such as Random, NN, Greedy Cycle and 2-regret. However, it is very important to choose the right weights for a given data set (problem).
  
- Greedy 2-regret heuristics outperforms only the random algorithm, moreover it has high volatility. Potentially, this could be due to the fact that the algorithm tries to avoid adding the cheapest nodes at the expense of those whose addition at a later stage may prove more expensive. However, in a problem of this nature where not every vertex needs to be added to the solution this is not the best approach. Nevertheless, it is noticeable that using the value of 2-regret in the greedy cycle that is the greedy heuristics algorithm with a weighted sum criterion described above improves the overall result.